### Portail meteo.data.gouv.fr - Téléchargement-affichage-Extraction des données MENSUELLES de Météo-France (Métropole & outre-mer)
Une connexion internet est nécessaire pour le téléchargement automatique des archives de données.
1) Téléchargement des fichiers et décompression automatique, pour différents postes, dans plusieurs départements si besoin
2) Tracé du graphique chronologique pour le paramètre Précipitations RR des postes choisis par l'utilisateur
3) Sauvegarde des données intégrales et du graphique dans un fichier Excel (tous paramètres pour tous les postes des départements concernés).

data : https://meteo.data.gouv.fr/ (6 min, horaire, quotidien, mensuel)<br>
Fiche PDF des postes : https://www.data.gouv.fr/fr/datasets/r/bee4b0c7-260a-40fe-b463-ed5631d6dc39 (paramètres et périodes de mesure)<br>
Fichier CSV descriptif champs: https://www.data.gouv.fr/fr/datasets/r/6d4ac560-8f7c-477f-9a3f-3c33137fc04e

Utilisez mon autre script pour visualiser sous forme de carte la liste des postes météorologiques fournie par Météo-France sous forme de fichier JSON https://meteo.data.gouv.fr/https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922

NB:
- Les historiques MENSUELS sont réparties en 3 fichiers pour chaque département: xxxx-1949, previous-1950-2022, latest-2023-2024 (LATEST indique le dernier fichier mis à jour quotidiennement depuis janvier de l'année précédente jusqu'au mois en cours, XXXX représent l'année de début de la période la plus ancienne variable selon le département).
- Les données MENSUELLES ne sont pas simplement équivalentes à des données QUOTIDIENNE agrégées (les paramètres ne sont pas strictement les mêmes. Par exemple, il existe des des NOMBRES DE JOURS DE PLUIE/GELEE/CHALEUR/etc.., ainsi que des précipitations MENSUELLES ESTIMEES 'RR_ME' plus anciennes que les mesures)

Auteur: https://github.com/loicduffar

##### 1) Lecture après téléchargement automatique des archives GZ et décompression des fichiers CSV pour tous les départements concernés

- Définir les chemins d'enregistrement pour les archives GZ et pour les fichier CSV décompressés
- Définir l'archive souhaitée (les historiques Météo-France sont partagés en plusieurs parties)
- Définir les postes à interroger par leur code (voir fichier "fiches.json" sur https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922)
- On peut choisir d'éviter de télécharger-décompresser les archives GZ si les fichiers CSV ont déjà été téléchargés lors du mois en cours

Les archives sont automatiquement téléchargées-décompressées, et les fichiers CSV sont lus


In [11]:
############ Auteur: L. Duffar ###########
############ Décembre 2023 ###########
# python 3.8.12
# Télécharge les les archives ZIP & et décompresse les fichiers CSV
import os
import requests
import pandas as pd
import numpy as np
import datetime
import time
import sys
import gzip

# ================ Personalisation ====================
# Chemin d'enregistrement local pour les archives gz et les fichiers CSV décompressés
folder_gz= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\archives\2023 Déc\base\M"
folder_csv= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M"

# Terminaison de l'archive mensuelle Météo-France, selon la période souhaitée (1 archive par période et par département dont le numéro sera ajouté automatiquement avant ce template)
# ATTENTION: tous ces noms sont variables selon le département et l'année en cours le jour de l'interrogation (ces variations ne sont que partiellement automatisées par le script à ce jour en décembre 2023)
# template_end= '_latest-2022-2023.csv' # année précédente et année en cours
template_end= '_previous-1950-2021.csv' # de 1950 à 2 ans avant l'année en cours
# template_end= '_xxxx-1949.csv' # xxxx sera automatiquement remplacé par l'année indiquée dans le dictionnaire "first_year" (début de la première période qui est variable selon le département)

# Numéro des Postes météo souhaités (chaine de 8 caractères) - voir fichier "fiches.json" sur https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922
# LES DEPARTEMENTS CORRESPONDANTS DOIVENT ËTRE PRESENTS DANS LA LISTE DES URLS plus bas après  ======= Initialisation ==========
postes= [
        # '04088001', '04039001', '04230001',                  # 04: Forcalquier, Castellane, Valensole \
        # '05046001',                                           # 05: Embrun \
        # '06088001', '06029001',                               # 06: Nice, Cannes \
        # '13001009', '13111002', '13103001','13055001',        # 13: Aix en Provence, Vauvenargue, Salon, Marseille, \
        # '83031001', '83061001', '83137001',                   # 83: Le Luc, Fréjus, Toulon \
        # '84003002', '84009002',                               # 84: Apt-Viton, Bastide des Jourdans \
        '97411132', '97411150',                               # 974 LA REUNION: Chaudron, Saint-François \
        '98511001',                                            # 985 MAYOTTE: Mamoudzou_SAPC \
        # '98404001', '98404002',                                 # 984 TERRES AUSTRALES ET ANTARTIQUES FRANCAISES: Kerguelen, Nouvelle Amsterdam \
        ]                               
# Choisir d'éviter le téléchargement/décompression
# non recommandé, sauf pour test ou à moins que le dernier téléchargement date du mois en cours
download= True

# ================ Extrait du fichier data CSV ============= (les colonnes sont en réalité séparées par des points-virgules)
# NUM_POSTE	NOM_USUEL	LAT	LON	ALTI	AAAAMM	RR	QRR	NBRR	RR_ME	RRAB	QRRAB	RRABDAT	NBJRR1	NBJRR5	NBJRR10	NBJRR30
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202201	3.6	1	31		1.4	1	4	1	0	0	0
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202202	26.1	28		24.7	1	14	2	1	1	0
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202203	6.6	1	31		3.6	1	30	2	0	0	0
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202204	27	1	30		17.6	1	23	4	2	1	0
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202205	26	1	31		16.8	1	8	3	2	1	0

# ================ Extrait du fichier CSV Descriptif de quelques paramètres (ceux les plus souvent utiles, précipitations et températures) =============(les colonnes sont en réalité séparées par ":")
# NUM_POSTE       	 numéro Météo-France du poste sur 8 chiffres
# NOM_USUEL       	 nom usuel du poste
# LAT             	 latitude, négative au sud (en degrés et millionièmes de degré)
# LON             	 longitude, négative à l’ouest de GREENWICH (en degrés et millionièmes de degré)
# ALTI            	 altitude du pied de l'abri ou du pluviomètre si pas d'abri (en m)
# AAAAMM          	 mois
# RR              	 cumul mensuel des hauteurs de précipitation (en mm et 1/10)
# QRR             	 code qualité de RR
# NBRR            	 nombre de valeurs présentes de hauteur de précipitation quotidienne
# RR_ME           	 cumul mensuel estimé des hauteurs de précipitation (en mm et 1/10)
# RRAB            	 précipitation maximale tombée en 24 heures au cours du mois
# QRRAB           	 code qualite de RRAB
# NBJRR1          	 nombre de jours avec RR ≥ 1.0 mm
# TX              	 moyenne mensuelle des températures maximales (TX) quotidiennes (en °C et 1/10)
# QTX             	 code qualité de TX
# TN              	 moyenne mensuelle des températures minimales (TN) quotidiennes (en °C et 1/10)
# QTN             	 code qualité de TN
# TM              	 moyenne mensuelle des (TN+TX)/2 quotidiennes (en °C et 1/10)
# QTM             	 code qualité de TM
# TMM             	 moyenne mensuelle des températures moyennes (TM) quotidiennes (en °C et 1/10)
# QTMM            	 code qualité du TMM
# TMMIN           	 minimum mensuel des moyennes (TN+TX)/2 quotidiennes (en °C et 1/10)
# QTMMIN          	 code qualité de TMMIN
# TMMAX           	 maximum mensuel des moyennes (TN+TX)/2 quotidiennes (en °C et 1/10) 
# QTMMAX          	 code qualité du TMMAX
# NBJGELEE        	 nombre de jours avec gelée

# ================ Initialisation ====================
now= datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M"))
# Structure du nom des fichiers de données mensuelles (1 fichier par département dont le numéro sera ajouté automatiquement au début et à la fin du template ci-dessous)
template_start= 'MENSQ_'
# Fin du nom dépendant de l'année en cours
# template_end= '_latest-' + str(now.year-1) + '-' + str(now.year) + '.csv'

# ---------------- urls de téléchargement des archives https://meteo.data.gouv.fr/ (Ajouter d'autres départements si besoin)
# valable à la date de décembre 2023 SOUS RESERVE DE MODIFICATION ULTERIEURE PAR METEO-FRANCE
url_liste_postes= "https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922"
url_desc_m = "https://www.data.gouv.fr/fr/datasets/r/6d4ac560-8f7c-477f-9a3f-3c33137fc04e"
# année de début des mesures par département (0 si pas d'archive avant 1950) Valable à ce jour en décembre 2023 
first_year= {'04': 1872, '05': 1877, '06': 1877, '13': 1786, '83': 1871, '84': 1874, '971': 1929, '972': 1905, '973': 1946, '974': 1900, '975': 1915, '984': 0, '985': 1935, 
             '986': 0, '987': 1853, '988': 1938} 
urls= dict()
urls['04']= {'old': "https://www.data.gouv.fr/fr/datasets/r/19d07805-7b29-4b90-ba44-a095d7b1a270", #  (début 1872)
             'previous': "https://www.data.gouv.fr/fr/datasets/r/881ac0db-e8d0-4f62-a185-9fd33d5d5819",
             'latest': "https://www.data.gouv.fr/fr/datasets/r/5c2d6f73-4162-4f36-bb41-98d7bc3e2ee1"} 
urls['05']= {'old': "https://www.data.gouv.fr/fr/datasets/r/2860cb56-5c15-4d8f-9835-f1707c7d2b11", #  (début 1877)
             'previous': "https://www.data.gouv.fr/fr/datasets/r/16014ec2-cd93-415e-a3b5-c2f868dfbee2",                                 
             'latest': "https://www.data.gouv.fr/fr/datasets/r/589b53fd-adef-44a4-b306-1dee1500d2d2"}
urls['06']= {'old': "https://www.data.gouv.fr/fr/datasets/r/0fee5242-b44b-428d-91ae-bcbecfcad002", #  (début 1877)
             'previous': "https://www.data.gouv.fr/fr/datasets/r/21ab9616-ccd7-4812-925e-1bf9c38511df",
             'latest': "https://www.data.gouv.fr/fr/datasets/r/f8bdc686-8c37-4ced-8c36-407148bca194"}
urls['13']= {'old': 'https://www.data.gouv.fr/fr/datasets/r/55a4c65d-ae4e-4b24-9794-a4de27ae498c', #  (début 1786)
             'previous': "https://www.data.gouv.fr/fr/datasets/r/acb2c523-fd78-478e-bb82-ef48da5ee0e4",
             'latest': 'https://www.data.gouv.fr/fr/datasets/r/78fe3a8a-7f32-44dd-a1b5-8acb841d6cc1'}
urls['83']= {'old': "https://www.data.gouv.fr/fr/datasets/r/4128f344-81af-4215-9e80-ba7f66e1a225", #  (début 1871)
             'previous': "https://www.data.gouv.fr/fr/datasets/r/7c614ae6-f619-4ee9-b9f2-94863c321d89",
             'latest': 'https://www.data.gouv.fr/fr/datasets/r/dc5ee912-e84a-4f15-bd08-7f326795ce41'}
urls['84']= {'old': "https://www.data.gouv.fr/fr/datasets/r/99ac95f3-135b-4373-98bb-defbb8761716", #  (début 1874)
             'previous': 'https://www.data.gouv.fr/fr/datasets/r/11b4f6ca-71eb-4865-b0e8-2901c1e83295',
             'latest': 'https://www.data.gouv.fr/fr/datasets/r/75526d6c-81d5-4aad-8961-e9fc2a43f7b5'}
urls['971']= {'old': "https://www.data.gouv.fr/fr/datasets/r/98908922-fe44-41f0-8a71-1c6d63b89afa", # Ile de la Guadeloupe (début 1929) + Saint Barthélémy/Saint-Martin (EN REALITE 977/978)
              'previous': "https://www.data.gouv.fr/fr/datasets/r/11c2ddb5-e338-4242-98ae-58fda9fc89ba",
              'latest': 'https://www.data.gouv.fr/fr/datasets/r/5674e562-8b88-40f1-9544-018d978c8ca1'}
urls['972']= {'old': "https://www.data.gouv.fr/fr/datasets/r/6a547ea1-0d4d-4d39-90f0-ab2b2600dbd8", # Ile de la Martinique (début 1905)
              'previous': "https://www.data.gouv.fr/fr/datasets/r/5f49b2d3-3b85-43e4-be4f-f990aec3a96c",
              'latest': 'https://www.data.gouv.fr/fr/datasets/r/548c4dcf-6f8e-4ac2-8c78-65646d015062'}
urls['973']= {'old': "https://www.data.gouv.fr/fr/datasets/r/22ed6e53-c638-4f98-aaa9-587a048eb3eb", # Guyane (début 1946)
              'previous': "https://www.data.gouv.fr/fr/datasets/r/24cb299e-5381-43bc-8cc8-31a8ae032ed7",
              'latest': 'https://www.data.gouv.fr/fr/datasets/r/eed1b6ab-bf91-4b3b-b2b1-dad212e6dc9a'}
urls['974']= {'old': "https://www.data.gouv.fr/fr/datasets/r/82e3d205-7bce-4834-a2c1-58d202fafd6c", # Ile de la Réunion (début 1900)
              'previous': "https://www.data.gouv.fr/fr/datasets/r/5999af04-0583-42ee-81ad-355601c4456f",
              'latest': 'https://www.data.gouv.fr/fr/datasets/r/50a0d0fb-d97c-4ca3-b308-def80d69416d'}
urls['975']= {'old': "https://www.data.gouv.fr/fr/datasets/r/3129b185-d6ab-4751-8956-10e441439605", # Saint-Pierre et Miquelon (début 1915)
             'previous': "https://www.data.gouv.fr/fr/datasets/r/c48a8a59-fa42-47e1-bb2f-d3f92eb8d2b6",
             'latest': 'https://www.data.gouv.fr/fr/datasets/r/86b3a033-b7be-4733-9203-cef708aad6a4'}
urls['984']= {'old': "",                                                                             # Terres australes et antarctiques françaises (pas d'archive à ce jour avant 1950)
             'previous': "https://www.data.gouv.fr/fr/datasets/r/796716d0-b71f-497a-a374-659f9ddc4e5c",
             'latest': 'https://www.data.gouv.fr/fr/datasets/r/3a82cde3-63aa-4b57-b8ba-84820e92c910'}
urls['985']= {'old': "https://www.data.gouv.fr/fr/datasets/r/a424db07-25db-46db-bdb2-5a7daff54191", # Mayotte (début 1935)
              'previous': "https://www.data.gouv.fr/fr/datasets/r/b24a5125-a5c2-4081-88d4-3967de9ab6d4",
              'latest': 'https://www.data.gouv.fr/fr/datasets/r/8c5daf91-4634-40d8-822d-27f1c04cf2b4'}
urls['986']= {'old': "",                                                                               # Wallis et Futuna (pas d'archive à ce jour avant 1950)
             'previous': "https://www.data.gouv.fr/fr/datasets/r/0f91a1e6-9a9a-43e1-8b11-87e36669e1e6",
             'latest': 'https://www.data.gouv.fr/fr/datasets/r/fee9cc39-9ddd-43bb-b8d1-8af4c619f9fa'}
urls['987']= {'old': "https://www.data.gouv.fr/fr/datasets/r/091daad0-aaaf-4a8e-9f6b-cb25e72becab", # Polynésie Française (début 1853)
              'previous': "https://www.data.gouv.fr/fr/datasets/r/37b40525-8361-4e72-8024-bd7d03ca4631",
              'latest': 'https://www.data.gouv.fr/fr/datasets/r/a8f13c11-7442-4997-8431-e7e25168aa8d'}
urls['988']= {'old': "https://www.data.gouv.fr/fr/datasets/r/6ad7de0f-a1f8-4adf-bc62-f7922b541dd5", # Nouvelle Calédonie (début 1938)
              'previous': "https://www.data.gouv.fr/fr/datasets/r/73592caf-3613-49f6-b1ef-0b9478140c99",
              'latest': 'https://www.data.gouv.fr/fr/datasets/r/eda2d9b4-d815-4d1d-8d70-edd54363a84e'}
urls['xx']= {'old': "",
             'previous': "",
             'latest': ''}

# ---------------- Définition des fonctions
def convert_to_date(aaaamm):
    return pd.to_datetime(str(aaaamm), format='%Y%m', errors='coerce')

def download_file(url, filename):
    # dans le string filename, chercher la partie avant l'extension .csv à l'aide du module os.path.splitext
    file= os.path.join(folder_gz, filename) + '.gz'
    print('Téléchargement: ', file)
    response = requests.get(url)
    if response.status_code == 200:
        with open(file, 'wb') as f:
            f.write(response.content)
    else:
        print("Fichier d'archive non présent à l'url habituelle: ", file)

def decompress_gz(filename):
    file= os.path.join(folder_gz, filename) + '.gz' 
    if os.path.exists(file):
        with gzip.open(file, 'rb') as f_in:
            file= os.path.join(folder_csv, os.path.splitext(filename)[0]) + '.csv'
            print('Décompression', file)
            with open(file, 'wb') as f_out:
                f_out.write(f_in.read())
    else:
        print("Fichier d'archive non trouvé: ", file)
        print("Téléchargez l'archive GZ, manuellement ou en modifiant la variable 'download', puis relancez le script")

def read_csv(filename):
    file= os.path.join(folder_csv, os.path.splitext(filename)[0]) + '.csv'
    if os.path.exists(file):
        print('Lecture: ', file)
        df= pd.read_csv(file, header=0, sep=";", dtype={"NUM_POSTE":str, 'AAAAMM':str}, parse_dates=['AAAAMM'], date_parser= convert_to_date)
    else:
        print("Fichier CSV non trouvé: ", file)
        print("Téléchargez l'archive GZ, manuellement ou en modifiant la variable 'download', puis relancez le script")
        sys.exit() # interrompt le script
    return df

# ================ Traitement ====================
# Lecture en ligne du fichier JSON de la liste des postes (fichier incomplet à ce jour en décembre 2023, mais seule liste disponible)
req= requests.get(url_liste_postes)
if req.status_code == 200:
    data_json= req.json()
else:
    print('la requête a échoué avec le code : ', req.status_codes)
    sys.exit() # interrompt le script
df_liste_postes = pd.DataFrame(data_json['features']).T
# Ajout de colonnes avec les champs de properties
df_liste_postes['lat'] = df_liste_postes['geometry'].apply(lambda x: x['coordinates'][1]).astype(float)
df_liste_postes['lon'] = df_liste_postes['geometry'].apply(lambda x: x['coordinates'][0]).astype(float)
df_liste_postes['nom_usuel']= df_liste_postes['properties'].apply(lambda x: x['NOM_USUEL'].strip())
df_liste_postes['num_poste']= df_liste_postes['properties'].apply(lambda x: x['NUM_POSTE'].strip())
df_liste_postes['commune'] = df_liste_postes['properties'].apply(lambda x: x['COMMUNE'].strip())
df_liste_postes['ficheClimComplete'] = df_liste_postes['properties'].apply(lambda x: x['ficheClimComplete']).astype(float)
df_liste_postes['ficheClimReduite'] = df_liste_postes['properties'].apply(lambda x: x['ficheClimReduite']).astype(float)
df_liste_postes['alti'] = df_liste_postes['properties'].apply(lambda x: x['ALTI'])
# supprime les colonnes inutiles et classe par numéro de poste (donc par département)
df_liste_postes.drop(['type', 'geometry', 'properties'], axis=1, inplace=True)
df_liste_postes.sort_values(by= ['num_poste'], inplace=True)

# Lit en ligne le fichier "MENSQ_descriptif_champs.csv" de description des champs et le stocke dans un dataframe pandas
# définit le dataframe pandas avec les 2 colonnes "param" et "name_long" pour la description des champs 
df_desc_m = pd.read_csv(url_desc_m, sep=":", header= None, index_col=0, names= ["param", "name_long", 'complement'], dtype={"param":str, "name_long":str, 'complement':str}, encoding= 'utf-8')
df_desc_m.index= df_desc_m.index.str.strip()
df_desc_m['name_long']= df_desc_m['name_long'].str.strip()

# fait la liste des départements concernés et garde les 2 premiers chiffres uniques
departements = []
for poste in postes:
    if float(poste[:2]) > 95:
        departements.append(poste[:3])
    else:
        departements.append(poste[:2])

# On ne garde que départements uniques
departements = list(set(departements)) # set() ignore automatiquement les doublons (ce qui évite de passer par array numpy pour utiliser la fonction np.unique() )
# On trie la liste
departements.sort()
# On affiche la liste
print('Départements concernés: ', departements)

# Téléchargement/décompression/lecture des fichiers dans une boucle sur les départements (urls tirées du dictionnaire 'urls')
i, j= 0, 0
for departement in departements:
    # url = urls[departement]            
    # Formation du nom du fichier à partir du template et du numéro de département (avec le cas particulier de la période la plus ancienne dont l'année de début dépend du département)
    # tester si xxxx est dans le string template_end
    if 'xxxx' in template_end:
        template_end= template_end.replace('xxxx', str(first_year[departement]))
    filename = f"{template_start}{departement}{template_end}"

    if 'latest' in template_end:
        url= urls[departement]['latest']
    elif 'previous' in template_end:
        url= urls[departement]['previous']
    else:
        url= urls[departement]['old']
    
    if url != '':   # si url non vide, on télécharge le fichier
        if download:
            download_file(url, filename)
            decompress_gz(filename)
        # Lecture du fichier CSV dans un dataframe pandas  
        df_departement= read_csv(filename)
        if i== 0: # pour le premier département, initialisation du dataframe final df
            df= df_departement 
            i= i+1
        else: # sinon concaténation du nouveau département dans llde dataframe df
            df= pd.concat([df, df_departement])        
    else:     # si url vide   
        print("Pas d'archive disponible pour ce département et cette période: ", filename)
        j= j+1
        if j== len(departements): # arrête l'exécution du script si l'archive était absente pour tous les départements
            sys.exit()
    
# supprime les espaces avant et après le numéro de poste
df['NUM_POSTE'] = df['NUM_POSTE'].str.strip()
# renome la colonne "AAAMM" en "DATE"
df.rename(columns={'AAAAMM':'DATE'}, inplace=True)
# supprime les espaces dans les noms de colonnes
df.columns= df.columns.str.strip()
display(df)

# Détection des postes manquants dans la liste JSON
postes_mesures= df['NUM_POSTE'].unique()
postes_json= df_liste_postes['num_poste'].unique()
# trouvrer les postes_mesures absents de la liste JSON
postes_absents= np.setdiff1d(postes_mesures, postes_json)
if len(postes_absents) > 0:
    # trouver le NOM_USEL des postes absents
    print("Postes des départements absents de la liste JSON:", postes_absents, df[df['NUM_POSTE'].isin(postes_absents)]['NOM_USUEL'].unique())
else:
    print("Aucun poste des départements absent de la liste JSON")


2023-12-31 13:24
Départements concernés:  ['974', '985']
Téléchargement:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\archives\2023 Déc\base\M\MENSQ_974_previous-1950-2021.csv.gz
Décompression X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M\MENSQ_974_previous-1950-2021.csv
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M\MENSQ_974_previous-1950-2021.csv
Téléchargement:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\archives\2023 Déc\base\M\MENSQ_985_previous-1950-2021.csv.gz
Décompression X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M\MENSQ_985_previous-1950-2021.csv
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M\MENSQ_

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,DATE,RR,QRR,NBRR,RR_ME,...,QNEIGETOTM,NEIGETOTAB,QNEIGETOTAB,NEIGETOTABDAT,NBJNEIGETOT1,NBJNEIGETOT10,NBJNEIGETOT30,NBJGREL,NBJORAG,NBJBROU
0,97401520,LE TEVELAVE,-21.211667,55.361333,908,1953-01-01,229.2,1.0,31.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97401520,LE TEVELAVE,-21.211667,55.361333,908,1953-02-01,105.5,1.0,28.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,97401520,LE TEVELAVE,-21.211667,55.361333,908,1953-03-01,440.9,1.0,31.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,97401520,LE TEVELAVE,-21.211667,55.361333,908,1953-04-01,364.8,1.0,30.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,97401520,LE TEVELAVE,-21.211667,55.361333,908,1953-05-01,152.0,1.0,31.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6590,98517001,COMBANI,-12.788833,45.135000,122,2021-08-01,3.5,1.0,31.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6591,98517001,COMBANI,-12.788833,45.135000,122,2021-09-01,31.7,1.0,30.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6592,98517001,COMBANI,-12.788833,45.135000,122,2021-10-01,117.9,1.0,31.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6593,98517001,COMBANI,-12.788833,45.135000,122,2021-11-01,287.9,1.0,30.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Postes des départements absents de la liste JSON: ['97402220' '97402250' '97403420' '97403440' '97403450' '97403460'
 '97403480' '97404520' '97404530' '97406240' '97407540' '97408510'
 '97408520' '97408530' '97408575' '97409220' '97409235' '97409250'
 '97409260' '97409270' '97410210' '97410218' '97410222' '97410226'
 '97410234' '97410244' '97410256' '97410262' '97410268' '97410274'
 '97410280' '97410286' '97411102' '97411103' '97411105' '97411108'
 '97411111' '97411114' '97411120' '97411126' '97411138' '97411141'
 '97411144' '97411160' '97412306' '97412312' '97412316' '97412318'
 '97412320' '97412324' '97412328' '97412332' '97412344' '97412348'
 '97412352' '97412360' '97412368' '97412372' '97412376' '97412380'
 '97413506' '97413512' '97413518' '97413524' '97413530' '97413536'
 '97413548' '97413554' '97413560' '97413562' '97413566' '97413572'
 '97413576' '97413584' '97413588' '97414408' '97414416' '97414422'
 '97414428' '97414434' '97414440' '97414446' '97414450' '97414456'
 '97414462' 

##### 2) Affichage des données mensuelles & graphique des précipitations
NB: 

- Définir les dates de début et de fin d'extraction comme suit:
    - None pour extraire la totalité des données de l'archive définie par l'utilisateur  dans la cellule 1. La fenêtre temporelle du graphique pourra toujours être modifiée à la souris, mais la péridoe d'extraction conditionne le début de la courbe de pluie cumulée (courbe qui peut être masquée à la souris)
    - ou définir la date de début et de fin, ce qui permet définit notamment le début de la courbe de pluie cumulée.
- Définir le ou les paramètres à extraire (au minimum les précipitations RR car le graphique est uniquement prévu pour cela)

In [12]:
# Graphique des précipitations avec la colonne 'DATE' en abscisse et la colonne 'RR' en ordonnée + Affichage des 2 dernière valeurs mensuelles
# chaque poste de la colonnee 'NUM_POSTE' est représenté dans un subplot séparé
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

#-------------- Personalisation du graphique 
# Paramètres à représenter: 'RR' obligatoire en premier, pour le graphique dont la présentation actuelle est spécialement adaptée aux précipitations
param_list= ['RR', 'TM', 'FXYAB']

# Période à représenter (None pour le début ou la fin du fichier par défaut)
start_date= None
# start_date= datetime.datetime(2022, 5, 1)

end_date= None
# end_date= datetime.datetime(now.year, now.month, now.day)

#-------------- initialisation des paramètres du graphique
param_RR= 'RR'
# Période par défaut if start_date= None ou  end_date= None
if start_date is None:
    start_date= df['DATE'].min()
if end_date is None:
    end_date= df['DATE'].max()

# période du GRAPHIQUE : TODO premier jour du premier mois au début du mois suivant (pour une graduation clair de l'axe des dates)
end_graph= end_date= df['DATE'].max()
# calcule start_graph  comme le premier jour de l'année précedent end_date
start_graph= datetime.datetime(end_graph.year-1, 1, 1)

# if now.month == 1:  # Si le mois actuel est janvier
#     start_graph = datetime.datetime(now.year - 1, 12, 1)
# else:  # Pour tous les autres mois
#     start_graph = datetime.datetime(now.year, now.month - 1, 1)

# filtre le dataframe sur les postes choisis
df_postes= df[df['NUM_POSTE'].isin(postes)]
# filtre le dataframe sur la période souhaitée
df_postes= df_postes[(df_postes['DATE'] >= start_date) & (df_postes['DATE'] <= end_date)]

#  Couleurs des courbes
color_sequence_bar = plotly.colors.qualitative.Set3
color_sequence_scatter = plotly.colors.qualitative.Pastel

# Axe des ordonnées secondaire (cumulé)
specs= [[{'secondary_y': True}]]*len(postes)

#=========================== Graphique
fig = make_subplots(rows= len(postes), cols= 1,# shared_xaxes= True, # shared_xaxes= True inutile grâce à  matches='x' dans update_xaxes qui fait la même chose sans supprimer la graduation des dates des subplots 
                    subplot_titles= [f"{poste} - {df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0]}" for poste in postes], specs= specs)

for i, poste in enumerate(postes):
    df_poste = df_postes[df_postes['NUM_POSTE'] == poste]
    fig.add_trace(go.Bar(x= df_poste['DATE'], y= df_poste[param_RR], 
                          # name= poste
                         name= df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0],
                         marker= dict(color=color_sequence_bar[i % len(color_sequence_bar)]),  # Utilisez la couleur correspondante de la séquence
                         ),
                row= i+1, col= 1)
    fig.add_trace(go.Scatter(x= df_poste['DATE'], y= df_poste[param_RR].cumsum(), 
                             name= f"Cumulé {poste}", yaxis= "y2",
                             line= dict(color= 'gray'),
                            #  line= dict(color= color_sequence_scatter[i % len(color_sequence_scatter)]), # sinon l'incrémentation dans la séquence de couleurs est de +2 à cause de la trace Bar
                             mode='lines' 
                             ), 
                 row=i+1, col=1, secondary_y= True)

fig.update_layout(title_text= 'meteo.data.gouv.fr - ' + "Précipitations mensuelles (mm) - " + now.strftime("%Y-%m-%d") , title_x= 0.5, 
                  height= 133.*(len(postes)+1), width= 1000,
                  hovermode= 'x unified', hoverlabel= dict(bgcolor='rgba(255,255,255,0.6)'),
                  )

# masque les traces scatter en laissant la possbilité de les afficher à nouveau, sauf pour la période est la plus récente LATEST
if 'latest' not in template_end: # si la période est la plus récente
    for i in range(1, len(postes) + 1):
        fig.update_traces(visible='legendonly', selector=dict(name=f"Cumulé {postes[i-1]}"))

# Survol par la souris : ligne verticale à travers tous les subplots matérialisant l'abscisse 
xlast= 'x' + str(len(postes))
# fig.update_traces(xaxis= xlast)  # xlast est le nom de l'axe des abscisses du dernier subplot

# Définition générale des axes
max_value= df_postes[param_RR].max()
max_value_cum= df_postes.groupby(['NUM_POSTE']).sum()[param_RR].max()
fig.update_yaxes(secondary_y= False, range= [0, max_value])  # Axe principale: Replace max_value with the desired maximum coordinate
fig.update_yaxes(secondary_y= True, title_text= "Cumulé", range= [0, max_value_cum]) # Axe secondaire: Replace max_value with the desired maximum coordinate

# Graduations de l'axe des DATES ...............................................................
if end_graph - start_graph > datetime.timedelta(days= 365): # Plus d'un an
    tickformat= '%b\n%Y'
    dtick= 'M6'
    showgrid_x= True
else: # Moins d'un an
    tickformat= '%b %Y'
    dtick= 'M1'
    showgrid_x= False
# ...............................Premiers subplots
# ................................................ en fonction de la longueur de la période
for i in range(1, len(postes) + 1):
    fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=i, col=1, matches='x')
# ...............................Dernier subplot
# ................................................ en fonction de la longueur de la période
fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=len(postes), col=1, matches='x', range=[start_graph, end_graph])
#                                                SANS tenir compte de la longueur de la période
# fig.update_xaxes(tickformat= '%b\n%Y', tickmode= 'linear', dtick= 'M1', row=len(postes), col=1, matches='x')

# GRILLE .............................................................................
# .................. verticales...
fig.update_xaxes(showgrid= showgrid_x, gridwidth= 1, gridcolor= 'white')
# .................. horizontales
#                                Axe secondaire:
fig.update_yaxes(secondary_y=True, showgrid=False) # pas de grille


#-------------- sauvegarde le graphique
file_graph= "meteo.data MENS"
fig.write_html(os.path.join(folder_csv, file_graph + template_end[:-4] + ".html"))
# sauvegarde le graphique dans un fichier png
fig.write_image(os.path.join(folder_csv, file_graph + template_end[:-4] + ".png"))

#=========================== Affichage des 2 dernières valeurs mensuelles
#-------------- Valeurs quotidiennes pivotée pour la présentation avec les postes en colonnes
#               pour les postes et les paramètres SELECTIONNES, SUR la durée COMPLETE
for i, par in enumerate(param_list):
    if i==0:
        df_m = df[df['NUM_POSTE'].isin(postes)].pivot(index='DATE', columns='NOM_USUEL', values= param_list[i]) \
            .assign(parametre= par)
    else:
        # append des lignes à df_m
        df_m = pd.concat([df_m,
                        df[df['NUM_POSTE'].isin(postes)].pivot(index='DATE', columns='NOM_USUEL', values= param_list[i]) \
                        .assign(parametre= par)]
                        )
df_m= df_m.reset_index().set_index(['parametre', 'DATE'], drop= True)

#-------------- Affichage
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
# imprimer un avertissement si la dernière date du dataframe est antérieure au dernier jour du mois en cours

titre= "Données mensuelles des 2 derniers mois de l'archive demandée et pour les paramètres choisis: "
titre= titre + 'Attention le cumul du dernier mois est incomplet à ce jour'
print(titre)
for i, par in enumerate(param_list):
    titre= '- ' + par + ': ' + df_desc_m.loc[par]['name_long']
    print(titre)
# month_before_now= datetime.datetime(now.year, now.month, 1) - datetime.timedelta(days= 30)
month_before= end_date - datetime.timedelta(days= 30)
display(df_m[(df_m.index.get_level_values(1) >= month_before) & (df_m.index.get_level_values(1) <= end_date)])

fig.show()

titre_m= "Données mensuelles de l'archive demandée complète pour les paramètres choisis: "
print(titre_m)
display(df_m)

2023-12-31 13:25
Données mensuelles des 2 derniers mois de l'archive demandée et pour les paramètres choisis: Attention le cumul du dernier mois est incomplet à ce jour
- RR: cumul mensuel des hauteurs de précipitation (en mm et 1/10)
- TM: moyenne mensuelle des (TN+TX)/2 quotidiennes (en °C et 1/10)
- FXYAB: maximum absolu mensuel de la force maximale quotidienne du vent moyenné sur 10 mn (FXY), à 10 m (en m/s et 1/10)


NOM_USUEL             CHAUDRON  MAMOUDZOU_SAPC  SAINT-FRANCOIS
parametre DATE                                                
RR        2021-11-01      17.0           118.5           116.8
          2021-12-01     145.6            81.3           148.7
TM        2021-11-01      25.7            27.7            21.9
          2021-12-01      27.2            28.4            23.5
FXYAB     2021-11-01       NaN             NaN             NaN
          2021-12-01       NaN             NaN             NaN

Données mensuelles de l'archive demandée complète pour les paramètres choisis: 


NOM_USUEL             CHAUDRON  MAMOUDZOU_SAPC  SAINT-FRANCOIS
parametre DATE                                                
RR        1953-03-01       NaN             NaN           240.1
          1953-04-01       NaN             NaN            85.6
          1953-05-01       NaN             NaN           208.8
          1953-06-01       NaN             NaN            57.9
          1953-07-01       NaN             NaN            57.5
...                        ...             ...             ...
FXYAB     2021-08-01       NaN             NaN             NaN
          2021-09-01       NaN             NaN             NaN
          2021-10-01       NaN             NaN             NaN
          2021-11-01       NaN             NaN             NaN
          2021-12-01       NaN             NaN             NaN

[2478 rows x 3 columns]

##### 3) Sauvegarde les données intégrales dans un fichier Excel (tous les paramètres pour tous les postes des départements concernés)
Le fichier est sauvegardé dans le même dossier local que les fichiers CSV décompressés

In [11]:
# Sauvegarde toutes les informations dans un fichier Excel
print(now.strftime("%Y-%m-%d %H:%M"))
print('Sauvegarde en cours, patientez...')
file= os.path.join(folder_csv, "meteo.data MENS" + '_' + '-'.join(departements) + template_end[:-4] + ".xlsx")
writer = pd.ExcelWriter(file, engine='xlsxwriter')
workbook  = writer.book
#======================== Sauvegarde la liste des postes dans le fichier excel

# sauvegarde dans un fichier excel plus lisible que le JSON après suppression des colonnes type, geometry et properties
sheet= 'liste_postes'
df_liste_postes.to_excel(writer, sheet_name= sheet, index= False, startrow= 0)

#======================== Sauvegarde la liste des champs dans le fichier Excel
sheet= 'MENSQ_descriptif_champs'
df_desc_m.to_excel(writer, sheet_name= sheet, index= True, startrow= 1)
worksheet = writer.sheets[sheet]
worksheet.write('A1', sheet + '.csv')
                
sheet= 'donnée complètes'
df.to_excel(writer, sheet_name= sheet, index=False, startrow= 2)
worksheet = writer.sheets[sheet]
worksheet.write('A1', now.strftime("%Y-%m-%d %H:%M"))
worksheet.write('A2', 'Données LATEST quotidiennes au format original, complètes pour tous les départements concernés: ')

#======================== Sauvegarde les dataframes dans le fichier Excel
sheet= 'donnée complètes'
df.to_excel(writer, sheet_name= sheet, index=False, startrow= 2)
worksheet = writer.sheets[sheet]
worksheet.write('A1', now.strftime("%Y-%m-%d %H:%M"))
worksheet.write('A2', "Données mensuelles LATEST (depuis le début de l'année précédente) au format original, complètes pour tous les départements concernés: ")

# ajoute un onglet exel avec df_m
n_param= len(param_list)
sheet= 'MENS Sélection'
df_m.to_excel(writer, sheet_name= sheet, index=True, startrow= n_param + 2)
worksheet = writer.sheets[sheet]
worksheet.write('A1', now.strftime("%Y-%m-%d %H:%M"))
worksheet.write('A2', titre_m)
for i, par in enumerate(param_list):
    titre= '- ' + par + ': ' + df_desc_m.loc[par]['name_long']
    worksheet.write('A' + str(i+3), titre)

# ajoute un onglet excel et enregistre une image statique du graphique
sheet= 'graph'
worksheet = workbook.add_worksheet(sheet)
worksheet.insert_image('A1', os.path.join(folder_csv, file_graph + template_end[:-4] + ".png"))

writer.save()

print(now.strftime("%Y-%m-%d %H:%M"))
print('Ficher Excel sauvegardé: ', file)

2023-12-29 11:00
Sauvegarde en cours, patientez...
2023-12-29 11:00
Ficher Excel sauvegardé:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M\meteo.data MENS_984_previous-1950-2021.xlsx
